In [1]:
!pip install mediapipe opencv-python tensorflow opencv-python


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import cv2
import numpy as np
import os
import mediapipe as mp
print(dir(mp.solutions.holistic))
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.callbacks import TensorBoard

['FACEMESH_CONTOURS', 'FACEMESH_TESSELATION', 'HAND_CONNECTIONS', 'HandLandmark', 'Holistic', 'NamedTuple', 'POSE_CONNECTIONS', 'PoseLandmark', 'SolutionBase', '_BINARYPB_FILE_PATH', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_download_oss_pose_landmark_model', 'constant_side_packet_calculator_pb2', 'detections_to_rects_calculator_pb2', 'download_utils', 'gate_calculator_pb2', 'image_to_tensor_calculator_pb2', 'inference_calculator_pb2', 'landmark_projection_calculator_pb2', 'local_file_contents_calculator_pb2', 'non_max_suppression_calculator_pb2', 'np', 'rect_transformation_calculator_pb2', 'roi_tracking_calculator_pb2', 'split_vector_calculator_pb2', 'ssd_anchors_calculator_pb2', 'switch_container_pb2', 'tensors_to_classification_calculator_pb2', 'tensors_to_floats_calculator_pb2', 'tensors_to_landmarks_calculator_pb2']


In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [15]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [16]:
def mediapipe_detection(image, model):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image_rgb)
    return image, results

In [17]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [18]:
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['Indian', 'Language', 'Welcome', 'Sign', 'Hello', 'Bye', 'He', 'She', 'deaf', 'Hearing', 'Teacher', 'Thank you', 'colours', 'black', 'white', 'red', 'green', 'blue', 'family', 'child', 'day', 'morning', 'afternoon', 'understand', 'we'])
#'Indian', 'Language', 'Welcome', 'Sign', 'Hello', 'Bye', 'He', 'She', 'deaf', 'Hearing', 'Teacher', 'Thank you', 'colours', 'black', 'white', 'red', 'green', 'blue', 'family', 'child', 'day', 'morning', 'afternoon', 'understand', 'we'
#new = np.array([])
#actions = np.append(actions,new)
no_sequences = 30
sequence_length = 30
start_folder = 91

In [26]:
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    if not os.path.exists(action_path):
        os.makedirs(action_path)

    dir_max = 0
    try:
        dir_max = np.max(np.array(os.listdir(action_path)).astype(int))
    except ValueError:
        pass

    for sequence in range(1, no_sequences + 1):
        try:
            os.makedirs(os.path.join(action_path, str(dir_max + sequence)))
        except FileExistsError:
            pass

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [28]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(start_folder, start_folder+no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [37]:
# Preprocess data
label_map = {label: num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            if os.path.exists(file_path):  # Check if the file exists
                res = np.load(file_path)
                window.append(res)
            else:
                print(f"File not found: {file_path}")
        sequences.append(window)
        labels.append(label_map[action])

In [38]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [39]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],1)

In [40]:
# Create model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(48, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(actions.shape[0], activation='softmax'))

In [41]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=50, callbacks=[tb_callback])

Epoch 1/50
88/88 [==============================] - 81s 874ms/step - loss: 3.2202 - categorical_accuracy: 0.0387
Epoch 2/50
88/88 [==============================] - 77s 870ms/step - loss: 3.1274 - categorical_accuracy: 0.0652
Epoch 3/50
88/88 [==============================] - 77s 875ms/step - loss: 2.9045 - categorical_accuracy: 0.1088
Epoch 4/50
88/88 [==============================] - 77s 872ms/step - loss: 2.6534 - categorical_accuracy: 0.1472
Epoch 5/50
88/88 [==============================] - 77s 879ms/step - loss: 2.3368 - categorical_accuracy: 0.1966
Epoch 6/50
88/88 [==============================] - 78s 881ms/step - loss: 2.1773 - categorical_accuracy: 0.2399
Epoch 7/50
88/88 [==============================] - 78s 891ms/step - loss: 1.9737 - categorical_accuracy: 0.2825
Epoch 8/50
88/88 [==============================] - 78s 892ms/step - loss: 1.8051 - categorical_accuracy: 0.3459
Epoch 9/50
88/88 [==============================] - 78s 885ms/step - loss: 1.7074 - categorical_

In [42]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

5/5 [==============================] - 1s 212ms/step


In [43]:
print("Multilabel Confusion Matrix:")
print(multilabel_confusion_matrix(ytrue, yhat))
print("Accuracy:", accuracy_score(ytrue, yhat))

Multilabel Confusion Matrix:
[[[141   0]
  [  0   6]]

 [[138   0]
  [  0   9]]

 [[142   0]
  [  0   5]]

 [[136   3]
  [  0   8]]

 [[143   0]
  [  0   4]]

 [[142   0]
  [  0   5]]

 [[143   0]
  [  4   0]]

 [[141   3]
  [  0   3]]

 [[138   1]
  [  1   7]]

 [[137   1]
  [  1   8]]

 [[140   0]
  [  0   7]]

 [[140   0]
  [  4   3]]

 [[144   0]
  [  1   2]]

 [[138   1]
  [  0   8]]

 [[138   1]
  [  1   7]]

 [[135   1]
  [  0  11]]

 [[142   1]
  [  0   4]]

 [[136   1]
  [  2   8]]

 [[141   0]
  [  2   4]]

 [[145   0]
  [  0   2]]

 [[145   0]
  [  0   2]]

 [[139   4]
  [  0   4]]

 [[144   0]
  [  0   3]]

 [[141   0]
  [  1   5]]

 [[142   0]
  [  0   5]]]
Accuracy: 0.8843537414965986


In [44]:
model.save('action.h5')

C:\Users\kesav\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('action.h5')

In [20]:
colors = [(245,117,16), (117,245,16), (16,117,245), (200,100,33), (22,122,222), (199,233,123), (111,222,123), (121,212,210), (140,160,180), (190,230,50), (109,39,22), (33,39,36), (44,24,144), (111,22,33),(245,117,16), (117,245,16), (16,117,245), (200,100,33), (22,122,222), (199,233,123), (111,222,123), (121,212,210), (140,160,180), (190,230,50), (109,39,22)]

In [21]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

In [22]:
def prob_viz(res, actions, image, colors):
    # This function should create a visualization of the probabilities
    # and display it on the image.
    # `res` is the result from the model prediction.
    # `actions` are the possible actions or classes.
    # `image` is the frame on which to draw.
    # `colors` is a list of colors to use for each action.

    # Example implementation:
    for num, prob in enumerate(res):
        cv2.rectangle(image, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(image, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return image

In [23]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.1) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]
            image = prob_viz(res, actions, image, colors)
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step
